In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline
import os

In [2]:
data = input_data.read_data_sets('data/fashion', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#training Set
print data.train.images.shape
print data.train.images.shape

#Test Set
print data.test.images.shape
print data.test.images.shape

(55000, 784)
(55000, 784)
(10000, 784)
(10000, 784)


In [4]:
label_dict = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot',
    
}

#Reshape Images
train_x = data.train.images.reshape(-1,28,28,1)
test_x = data.test.images.reshape(-1,28,28,1)

train_y = data.train.labels
test_y = data.test.labels

In [5]:
train_iter = 5
lr = 0.001
bs = 128

input_number = 28
classes = 10

In [6]:
x=tf.placeholder("float",[None, 28,28,1])
y=tf.placeholder("float",[None, classes])

In [7]:
def conv2d(x, w, b, strides = 1):
    #bias and relu activation
    x = tf.nn.conv2d(x,w,strides=[1, strides, strides,1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,k,k,1],padding='SAME')


In [8]:
#weights and biases defination

weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()),
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer= tf.contrib.layers.xavier_initializer()),
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()),
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('W6', shape=(128,classes), initializer=tf.contrib.layers.xavier_initializer()),
}

biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer= tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer= tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer= tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer= tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(10), initializer= tf.contrib.layers.xavier_initializer()),
    
}

In [9]:
def conv_net(n, weights, biases):
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=2)
    
    #fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]]) 
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    #output, class prediciton
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
    
    
    

In [10]:
#define optimizer and cost function
pred = conv_net(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))

#calculate the accuracy and avg them out
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

In [12]:
#session preperation
sess = tf.InteractiveSession()
#initialize the variable
init = tf.global_variables_initializer()
sess.run(init)

In [13]:

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []
summary_writer = tf.summary.FileWriter('./Output', sess.graph)
for i in range(train_iter):
    for batch in range(len(train_x)//bs):
        batch_x = train_x[batch*bs:min((batch+1)*bs, len(train_x))]
        batch_y = train_y[batch*bs:min((batch+1)*bs, len(train_y))]
        #run optimization backprop and calculate loss and accuracy for batch
        opt = sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        loss, acc = sess.run([cost,accuracy], feed_dict={x:batch_x, y:batch_y})
    print "iter "+ str(i) +", Loss= "+ "{:.6f}".format(loss)+ ", Training accuracy= "+"{:.5f}".format(acc)
    print "optimization completed"

    #calculate the accuracy for 1000 test images
    test_acc, vaild_loss = sess.run([accuracy,cost], feed_dict={x: test_x, y:test_y})
    train_loss.append(loss)
    test_loss.append(vaild_loss)
    train_accuracy.append(acc)
    test_accuracy.append(test_acc)
    print "test accuracy" , "{:.5f}".format(test_acc)
    

iter 0, Loss= 0.053984, Training accuracy= 0.98438
optimization completed
test accuracy 0.95430
iter 1, Loss= 0.005158, Training accuracy= 1.00000
optimization completed
test accuracy 0.97650
iter 2, Loss= 0.001393, Training accuracy= 1.00000
optimization completed
test accuracy 0.98740
iter 3, Loss= 0.001149, Training accuracy= 1.00000
optimization completed
test accuracy 0.98710
iter 4, Loss= 0.002102, Training accuracy= 1.00000
optimization completed
test accuracy 0.98710
